In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_rop',
    'wp_sale_orders',
    proc_date,
    'temp_rop_wp_sale_orders'
)

In [0]:
spark.sql(f""" CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_rop.wp_sale_orders LIKE {catalog_name}.udp_wcm_bronze_rop.wp_sale_orders""")

In [0]:
spark.sql(f"""ALTER TABLE {catalog_name}.udp_wcm_silver_rop.wp_sale_orders
SET TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
          MERGE INTO {catalog_name}.udp_wcm_silver_rop.wp_sale_orders main_tbl
USING (
    SELECT source_client_id
        ,channel
        ,order_code
        ,cart_id
        ,status_name
        ,site_code
        ,product_barcode
        ,unit_code
        ,unit_price
        ,order_quanity as order_quantity
        ,delivered_quantity
        ,ordered_amount
        ,delivered_amount
        ,discount_per_unit as discount_per_amount
        ,net_order_amount
        ,vat
        ,is_pup_order
        ,customer_phone
        ,proc_date
            , FARM_FINGERPRINT(CONCAT(
                IFNULL(CAST(cart_id AS STRING), ""), 
                IFNULL(CAST(product_barcode AS STRING), ""),
                IFNULL(CAST(complete_date AS STRING), ""),
                IFNULL(CAST(status_name AS STRING), ""),
                IFNULL(CAST(site_code AS STRING), ""),
                IFNULL(CAST(unit_code AS STRING), ""),
                IFNULL(CAST(proc_date AS STRING), "")
            )) hash_id
            , to_timestamp(order_date, 'yyyyMMddHHmmss') as order_date
            , to_timestamp(complete_date, 'yyyyMMddHHmmss') as complete_date
            , LTRIM('0', CAST(article_id AS STRING)) article_id
            , to_timestamp(created_date, 'yyyyMMddHHmmss') as created_date
            , pos_code
            ,file_creation_ts
    FROM temp_rop_wp_sale_orders
) tmp_tbl
ON main_tbl.hash_id = tmp_tbl.hash_id 
WHEN NOT MATCHED THEN INSERT *
""")